### 1. Импорт библиотек и подготовка датафрейма:

In [1]:
from faker import Faker
import pandas as pd
import random
from gender_guesser.detector import Detector
from datetime import datetime

In [2]:
# Инициализируем генератор фейковых данных
fake = Faker()

# Инициализируем детектор пола
detector = Detector()

# Создаем пустой список для хранения данных
data = []

# Ввод пользователем количество людей и граница возрастов
num_people = int(input("Введите количество людей в списке: "))
minimum = int(input("Введите минимальный рабочий возраст: "))
maximum = int(input("Введите максимальный рабочий возраст: "))

# Создаем массив
for _ in range(num_people):
    full_name = fake.name().replace(' ', '_')
    first_name = full_name.split('_')[0]
    birthday = fake.date_of_birth(minimum_age=minimum, maximum_age=maximum)
    age = datetime.now().year - birthday.year
    gender = detector.get_gender(first_name)
    profession = fake.job()
    # Удаляем всё после запятой и саму запятую в профессии
    profession = profession.split(',')[0]
    monthly_salary = random.randint(20000, 500000)
    yearly_salary = monthly_salary * 12

    # Добавляем данные в список
    data.append([full_name, birthday, age,
                 gender, profession,
                 monthly_salary, yearly_salary])

# Создаем датафрейм
df = pd.DataFrame(data, columns=['name', 'birthday', 'age',
                                 'gender', 'profession',
                                 'monthly_salary', 'yearly_salary'])

# Заменяем 'mostly_male' и 'mostly_female' на 'male' и 'female'
df['gender'] = df['gender'].replace({'mostly_male':'male', 'andy':'male', 'mostly_female':'female'})
df['birthday'] = pd.to_datetime(df['birthday'])

# Заменяем gender == unknown на male для имен, начинающихся с 'Dr.' и 'Mr.', и на female для имен, начинающихся с 'Mrs.'
df.loc[df['name'].str.startswith('Dr.') |
       df['name'].str.startswith('Mr.'), 'gender'] = 'male'

df.loc[df['name'].str.startswith('Mrs.') |
       df['name'].str.startswith('Latoya') |
       df['name'].str.startswith('Ms.') |
       df['name'].str.startswith('Miss'), 'gender'] = 'female'

# Выводим обновленный датафрейм
df

Введите количество людей в списке:  1000000
Введите минимальный рабочий возраст:  22
Введите максимальный рабочий возраст:  68


,name,birthday,age,gender,profession,monthly_salary,yearly_salary
0,Bethany_Garcia,1961-12-16,63,female,Jewellery designer,130242,1562904
1,Aaron_Crawford,1970-06-16,54,male,Transport planner,41835,502020
2,Cole_Kim,1960-07-25,64,male,General practice doctor,284853,3418236
3,Brett_Frazier,1975-11-18,49,male,Music therapist,360209,4322508
4,Mark_Harvey,1994-11-01,30,male,Hydrographic surveyor,267521,3210252
...,...,...,...,...,...,...,...
999995,Jessica_Thompson,1975-09-08,49,female,Estate agent,57074,684888
999996,Heather_Thompson,1996-02-25,28,female,Illustrator,359833,4317996
999997,Brittany_Rivera,1965-04-26,59,female,Teacher,496135,5953620
999998,Dr._Jeremy_Carter_MD,1960-02-29,64,male,Magazine journalist,238209,2858508


Количество уникальных профессий в датафрейме:

In [3]:
df.profession.nunique()

520

Количество мужчин и женщин в датасете:

In [4]:
df['gender'].value_counts()

gender
male      511609
female    488391
Name: count, dtype: int64

### 2. Задания к датафрейму:

1) Наиболее распространенные профессии (топ-10)
2) Наиболее распространенные профессии среди мужчин (топ-10)
3) Наиболее распространенные профессии среди женщин (топ-10)
4) Наиболее прибыльные профессии в среднем (топ-10)
5) Наиболее прибыльные профессии в среднем среди мужчин (топ-10)
6) Наиболее прибыльные профессии в среднем среди женщин (топ-10)
7) Топ-10 профессий с самым большим средним возрастом
8) Топ-10 профессий с самым маленьким средним возрастом
9) Года в котором родилось наибольшее количество людей (топ-10)
10) Года в котором родилось наименьшее количество людей (топ-10)

#### 1) Наиболее распространенные профессии (топ-10)

In [5]:
top_professions = df['profession'].value_counts().reset_index().head(10)
top_professions

,profession,count
0,Engineer,43997
1,Scientist,18740
2,Designer,18588
3,Surveyor,17100
4,Therapist,12213
5,Psychologist,11103
6,Administrator,10996
7,Teacher,10954
8,Accountant,6291
9,Nurse,6106


#### 2) Наиболее распространенные профессии среди мужчин (топ-10)

In [6]:
top_professions = (df.query('gender == "male"')['profession'].value_counts()
                     .reset_index()
                     .head(10))
top_professions

,profession,count
0,Engineer,22463
1,Scientist,9699
2,Designer,9467
3,Surveyor,8765
4,Therapist,6165
5,Psychologist,5774
6,Administrator,5620
7,Teacher,5512
8,Accountant,3203
9,Nurse,3168


#### 3) Наиболее распространенные профессии среди женщин (топ-10)

In [7]:
top_professions = (df.query('gender == "female"')['profession'].value_counts()
                     .reset_index()
                     .head(10))
top_professions

,profession,count
0,Engineer,21534
1,Designer,9121
2,Scientist,9041
3,Surveyor,8335
4,Therapist,6048
5,Teacher,5442
6,Administrator,5376
7,Psychologist,5329
8,Accountant,3088
9,Nurse,2938


#### 4) Наиболее прибыльные профессии в среднем (топ-10)

In [8]:
(df.groupby('profession')['yearly_salary'].mean()
    .reset_index()
    .sort_values(by='yearly_salary', ascending=False)
    .head(10))

,profession,yearly_salary
144,Diagnostic radiographer,3.228707e+06
470,Surgeon,3.214534e+06
446,Seismic interpreter,3.210983e+06
178,Event organiser,3.210414e+06
26,Animal nutritionist,3.210355e+06
323,Multimedia specialist,3.202461e+06
20,Air cabin crew,3.202081e+06
334,Neurosurgeon,3.201223e+06
76,Chartered accountant,3.200479e+06
408,Publishing rights manager,3.200151e+06


#### 5) Наиболее прибыльные профессии в среднем среди мужчин (топ-10)

In [9]:
(df.query('gender == "male"')
    .groupby('profession')['yearly_salary'].mean()
    .reset_index()
    .sort_values(by='yearly_salary', ascending=False)
    .head(10))

,profession,yearly_salary
470,Surgeon,3.267592e+06
26,Animal nutritionist,3.255069e+06
167,English as a foreign language teacher,3.251812e+06
269,Investment analyst,3.242139e+06
386,Producer,3.238546e+06
220,Glass blower/designer,3.235289e+06
513,Waste management officer,3.233729e+06
323,Multimedia specialist,3.232605e+06
144,Diagnostic radiographer,3.232370e+06
431,Retail manager,3.232010e+06


#### 6) Наиболее прибыльные профессии в среднем среди женщин (топ-10)

In [10]:
(df.query('gender == "female"')
    .groupby('profession')['yearly_salary'].mean()
    .reset_index()
    .sort_values(by='yearly_salary', ascending=False)
    .head(10))

,profession,yearly_salary
4,Actor,3.283784e+06
420,Recruitment consultant,3.276987e+06
314,Merchant navy officer,3.276111e+06
506,Tree surgeon,3.271028e+06
408,Publishing rights manager,3.262471e+06
182,Facilities manager,3.261646e+06
37,Armed forces technical officer,3.256214e+06
283,Lexicographer,3.254834e+06
334,Neurosurgeon,3.254775e+06
230,Herbalist,3.253142e+06


#### 7) Топ-10 профессий с самым большим средним возрастом

In [11]:
(df.groupby('profession')['age'].mean()
    .reset_index()
    .sort_values(by='age', ascending=False)
    .head(10))

,profession,age
4,Actor,46.923639
310,Medical secretary,46.780839
138,Data scientist,46.744535
244,Housing manager/officer,46.742747
430,Retail buyer,46.688062
402,Public affairs consultant,46.658934
232,Herpetologist,46.619201
236,Holiday representative,46.617628
497,Town planner,46.605131
88,Chief Strategy Officer,46.572554


#### 8) Топ-10 профессий с самым маленьким средним возрастом

In [12]:
(df.groupby('profession')['age'].mean()
    .reset_index()
    .sort_values(by='age', ascending=True)
    .head(10))

,profession,age
318,Midwife,44.869478
234,Higher education lecturer,44.988024
78,Chartered legal executive (England and Wales),45.065531
411,Quantity surveyor,45.075982
325,Museum/gallery conservator,45.085411
331,Nature conservation officer,45.090617
175,Ergonomist,45.129987
300,Market researcher,45.174812
488,Theatre director,45.188320
109,Commercial/residential surveyor,45.230461


#### 9) Года в котором родилось наибольшее количество людей (топ-10)

In [13]:
# Посчитаем, сколько людей родилось в каждом году
birth_year_counts_more = (
    df['birthday']
    .dt.year
    .value_counts()
    .reset_index()
    .sort_values(by='count', ascending=False)
    .head(10)
)

# Топ-10 годов, в которых родилось наибольшее количество людей
birth_year_counts_more

,birthday,count
0,1994,21596
1,1988,21564
2,1956,21536
3,1992,21513
4,1986,21505
5,1982,21425
6,1957,21393
7,1973,21387
8,1996,21372
9,2001,21372


#### 10) Года в котором родилось наименьшее количество людей (топ-10)

In [14]:
# Посчитаем, сколько людей родилось в каждом году
birth_year_counts_less = (
    df['birthday']
    .dt.year
    .value_counts()
    .reset_index()
    .sort_values(by='count', ascending=True)
    .head(10)
)

# Топ-10 годов, в которых родилось наибольшее количество людей
birth_year_counts_less

,birthday,count
47,2002,2635
46,1955,18771
45,1979,21035
44,1970,21087
43,1998,21098
42,1984,21100
41,1967,21100
40,1975,21107
39,1987,21113
38,1991,21119
